# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '../../../../src/')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-sport5-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA finals explorative search") 

Consider the following exploratory information need:

> investigate the world of basketball. Compare italian teams and US teams, country of citizenship of NBA players, and teams which won NBA finals.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36159`| Lebron James      | node |
| `wd:Q29545`| Kevin Durant      | node |





Also consider

```
wd:Q36159 ?p ?obj .
```

is the BGP to retrieve all **properties of Lebron James**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Identify the BGP for NBA finals

6. Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

8. Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

9. Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)



In [3]:
## startup the evaluation
# setup the file and create the empty json
ipname = "s5.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 0_1


## Task 1 
Identify the BGP for basketball player

In [2]:
# starting from Lebron James I look at his properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    wd:Q36159 ?pr ?obj.
    ?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('pr', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('pr', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('pr', 'http://www.wikidata.org/prop/dir

In [3]:
# starting from Lebron James I look at his properties
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q36159 wdt:P106 ?obj.
    ?obj sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
3


In [4]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q3665646"
og_name = "basketball player"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_1.json
JSON object updated


`wd:Q3665646` is the basketball player class

## Task 2

In [4]:
# starting from Basketball player I look at its properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    wd:Q3665646 ?pr ?obj.
    ?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('pr', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('pr', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('pr', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('pr', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('pr', '

In [5]:
# starting from Basketball player I look at its properties
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q3665646 wdt:P641 ?obj.
    ?obj sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
1


In [5]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q5372"
og_name = "basketball"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_1.json
JSON object updated


`wd:Q5372` is the node of basketball

## Task 3

In [6]:
# starting from Lebron James I look at his properties
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q36159 wdt:P54 ?obj.
    ?obj sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q169138'), ('name', 'Miami Heat')]
[('obj', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
[('obj', 'http://www.wikidata.org/entity/Q162990'), ('name', 'Cleveland Cavaliers')]
3


In [7]:
# starting from Lebron James I look at his properties
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q36159 wdt:P54 ?obj.
    ?obj wdt:P31 ?inst.
    ?inst sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
1


In [6]:
### insert the result of TASK 3 in the file
og_uri = "http://www.wikidata.org/entity/Q13393265"
og_name = "basketball team"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_1.json
JSON object updated


`Q13393265` is the basketball team

## Task 4

In [8]:
# starting from Lebron James I look at his properties
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q36159 wdt:P54 ?obj.
    ?obj wdt:P17 ?inst.
    ?inst sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
1


In [9]:
# starting from Lebron James I look at his properties
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q30 wdt:P31 ?inst.
    ?inst sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q1489259'), ('name', 'superpower')]
[('inst', 'http://www.wikidata.org/entity/Q99541706'), ('name', 'historical unrecognized state')]
[('inst', 'http://www.wikidata.org/entity/Q1520223'), ('name', 'constitutional republic')]
[('inst', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('inst', 'http://www.wikidata.org/entity/Q512187'), ('name', 'federal republic')]
[('inst', 'http://www.wikidata.org/entity/Q5255892'), ('name', 'democratic republic')]
[('inst', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
7


In [10]:
# starting from countries I will try to find Italy
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    ?inst wdt:P31 wd:Q6256.
    ?inst sc:name ?name .
    FILTER (REGEX(?name,"tal")).
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
1


In [8]:
# starting from Lebron James I look at his properties
queryString = """
SELECT DISTINCT ?obj ?name year(?inception) as ?year
WHERE { 
    ?obj wdt:P31 wd:Q13393265.
    ?obj wdt:P17 wd:Q38.
    ?obj sc:name ?name .
    ?obj wdt:P571 ?inception.
    FILTER(year(?inception) < 1945).
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q36264'), ('name', 'Virtus Pallacanestro Bologna'), ('year', '1927')]
[('obj', 'http://www.wikidata.org/entity/Q36311'), ('name', 'Pallacanestro Cantù'), ('year', '1936')]
[('obj', 'http://www.wikidata.org/entity/Q961782'), ('name', 'Pallacanestro Petrarca Padova'), ('year', '1934')]
[('obj', 'http://www.wikidata.org/entity/Q36362'), ('name', 'Olimpia Milano'), ('year', '1936')]
[('obj', 'http://www.wikidata.org/entity/Q36500'), ('name', 'Mens Sana Basket'), ('year', '1861')]
[('obj', 'http://www.wikidata.org/entity/Q3891980'), ('name', 'Nuova Pallacanestro Pavia'), ('year', '1943')]
[('obj', 'http://www.wikidata.org/entity/Q1139737'), ('name', 'Fortitudo Pallacanestro Bologna'), ('year', '1932')]
[('obj', 'http://www.wikidata.org/entity/Q3754192'), ('name', 'Fulgor Omegna'), ('year', '1937')]
[('obj', 'http://www.wikidata.org/entity/Q7127695'), ('name', 'Pallacanestro Pavia'), ('year', '1943')]
[('obj', 'http://www.wikidata.org/entity/Q

In [9]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    year = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= year
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_1.json
JSON object updated


## Task 5

In [12]:
# starting from Lebron James I look at his properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    wd:Q36159 wdt:P54 ?obj.
    ?x ?pr ?obj.
    ?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P6291'), ('name', 'advertises')]
[('pr', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('pr', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P137'), ('name', 'operator')]
[('pr', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('pr', 'http://www.wikidata.org/prop/direct/P463'), ('name', 'member of')]
[('pr', 'http://www.wikidata.org/prop/direct/P466'), ('name', 'occupant')]
[('pr', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('pr', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]
[('pr', 'http://www.wikidata.org/prop/direct/P6087'), ('name', 'coach of sports team')]
[('pr', 'http://www.wikidata.org/prop/direct/P710'), ('name', 'participant')]
[('pr', 'http://www.wikidata.org/prop/direct/P74

In [13]:
# starting from Lebron James I look at his properties
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q36159 wdt:P54 ?obj.
    ?x wdt:P1346 ?obj.
    ?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q2294127'), ('name', '1971–72 NBA season')]
[('x', 'http://www.wikidata.org/entity/Q113619'), ('name', '2012–13 NBA season')]
[('x', 'http://www.wikidata.org/entity/Q19862716'), ('name', '2015–16 NBA season')]
[('x', 'http://www.wikidata.org/entity/Q631240'), ('name', '1979–80 NBA season')]
[('x', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
[('x', 'http://www.wikidata.org/entity/Q1142320'), ('name', '2005–06 NBA season')]
[('x', 'http://www.wikidata.org/entity/Q4566514'), ('name', '1949 BAA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4566866'), ('name', '1950 NBA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4575074'), ('name', '1972 NBA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4586947'), ('name', "1991 McDonald's Championship")]
[('x', 'http://www.wikidata.org/entity/Q4598928'), ('name', '2001 NBA Finals')]
[('x', 'http:

In [14]:
# starting from Lebron James I look at his properties
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q644353 wdt:P31 ?x.
    ?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q842375'), ('name', 'NBA Finals')]
1


In [10]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q842375"
og_name = "NBA Finals"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_1.json
JSON object updated


`wd:Q842375` is the NBA finals

## Task 6

In [9]:
# print NBA Finals
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    ?x wdt:P31 wd:Q842375.
    ?x sc:name ?name .
}
ORDER BY ?name
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q3072443'), ('name', '1947 BAA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4566218'), ('name', '1948 BAA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4566514'), ('name', '1949 BAA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4566866'), ('name', '1950 NBA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4567168'), ('name', '1951 NBA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4567490'), ('name', '1952 NBA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4567780'), ('name', '1953 NBA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4568101'), ('name', '1954 NBA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4568407'), ('name', '1955 NBA World Championship Series')]
[('x', 'http://www.wikidata.org/entity/Q4568729'), ('name', '1956 NBA World Championship Series')]
[(

In [16]:
# starting from Lebron James I look at his properties 
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    VALUES ?player {wd:Q36159 wd:Q29545}.
    ?player wdt:P54 ?obj.
    ?obj sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q180950'), ('name', 'Oklahoma City Thunder')]
[('obj', 'http://www.wikidata.org/entity/Q235326'), ('name', 'Seattle SuperSonics')]
[('obj', 'http://www.wikidata.org/entity/Q572134'), ('name', 'Brooklyn Nets')]
[('obj', 'http://www.wikidata.org/entity/Q169138'), ('name', 'Miami Heat')]
[('obj', 'http://www.wikidata.org/entity/Q7707830'), ('name', "Texas Longhorns men's basketball")]
[('obj', 'http://www.wikidata.org/entity/Q244847'), ('name', "United States men's national basketball team")]
[('obj', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
[('obj', 'http://www.wikidata.org/entity/Q157376'), ('name', 'Golden State Warriors')]
[('obj', 'http://www.wikidata.org/entity/Q162990'), ('name', 'Cleveland Cavaliers')]
9


In [11]:
# starting from Lebron James I look at his properties 
queryString = """
SELECT DISTINCT ?obj ?name COUNT(?x) AS ?finals
WHERE { 
    VALUES ?player {wd:Q36159 wd:Q29545}.
    ?player wdt:P54 ?obj.
    optional{?x wdt:P1346 ?obj.
    ?x wdt:P31 wd:Q842375.}
    ?obj sc:name ?name .
}
GROUP BY ?obj ?name
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q162990'), ('name', 'Cleveland Cavaliers'), ('finals', '0')]
[('obj', 'http://www.wikidata.org/entity/Q572134'), ('name', 'Brooklyn Nets'), ('finals', '0')]
[('obj', 'http://www.wikidata.org/entity/Q180950'), ('name', 'Oklahoma City Thunder'), ('finals', '0')]
[('obj', 'http://www.wikidata.org/entity/Q7707830'), ('name', "Texas Longhorns men's basketball"), ('finals', '0')]
[('obj', 'http://www.wikidata.org/entity/Q157376'), ('name', 'Golden State Warriors'), ('finals', '4')]
[('obj', 'http://www.wikidata.org/entity/Q235326'), ('name', 'Seattle SuperSonics'), ('finals', '1')]
[('obj', 'http://www.wikidata.org/entity/Q244847'), ('name', "United States men's national basketball team"), ('finals', '0')]
[('obj', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers'), ('finals', '16')]
[('obj', 'http://www.wikidata.org/entity/Q169138'), ('name', 'Miami Heat'), ('finals', '2')]
9


In [12]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    finals = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= finals
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_1.json
JSON object updated


## Task 7

In [18]:
# starting from Lebron James I look at his properties 
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    wd:Q36159 wdt:P27 ?country.
    ?country ?pr ?continent.
    ?pr sc:name ?name .
    filter(regex(?name,"contine")).
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
1


In [19]:
# starting from Lebron James I look at his properties 
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    VALUES ?player {wd:Q36159 wd:Q29545}.
    ?player wdt:P54 ?obj.
    ?obj ?pr ?x.
    ?pr sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('pr', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('pr', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('pr', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('pr', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('pr', 'http://www.wikidata.org/prop/direct/P1366'), ('name', 'replaced by')]
[('pr', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('pr', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('pr', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('name', 

In [20]:
# starting from Lebron James I look at his properties 
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    VALUES ?player {wd:Q36159 wd:Q29545}.
    ?player wdt:P54 ?obj.
    ?obj wdt:P118 ?x.
    ?x sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q94861615'), ('name', "NCAA Division I men's basketball")]
[('x', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
[('x', 'http://www.wikidata.org/entity/Q858430'), ('name', 'Big 12 Conference')]
3


In [13]:
# starting from Lebron James I look at his properties 
queryString = """
SELECT DISTINCT ?continent ?name COUNT(DISTINCT ?player) AS ?players
WHERE { 
    
    ?team wdt:P118 wd:Q155223.
    ?player wdt:P54 ?team;
        wdt:P27 ?country.
    ?country wdt:P30 ?continent.
    
    ?continent sc:name ?name.
}
GROUP BY ?continent ?name
"""

print("Results")
x = run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America'), ('players', '37')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia'), ('players', '73')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania'), ('players', '3799')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa'), ('players', '61')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America'), ('players', '3849')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia'), ('players', '17')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe'), ('players', '304')]
7


In [14]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    players = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= players
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"7", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_1.json
JSON object updated


## Task 8

In [22]:
# starting from Lebron James I look at his properties 
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    ?team wdt:P118 wd:Q155223.
    ?player wdt:P54 ?team.
    ?x wdt:P1346 ?player.
    
    ?x sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q1501719'), ('name', 'J. Walter Kennedy Citizenship Award')]
[('x', 'http://www.wikidata.org/entity/Q680221'), ('name', 'Laureus World Sports Award for Breakthrough of the Year')]
[('x', 'http://www.wikidata.org/entity/Q5593890'), ('name', 'Grammy Award for Best Spoken Word Album')]
[('x', 'http://www.wikidata.org/entity/Q4896605'), ('name', 'Best Championship Performance ESPY Award')]
[('x', 'http://www.wikidata.org/entity/Q918054'), ('name', 'NBA Coach of the Year Award')]
[('x', 'http://www.wikidata.org/entity/Q4896598'), ('name', 'Best Breakthrough Athlete ESPY Award')]
[('x', 'http://www.wikidata.org/entity/Q1418596'), ('name', 'NBA Sportsmanship Award')]
[('x', 'http://www.wikidata.org/entity/Q1035067'), ('name', 'Heisman Trophy')]
[('x', 'http://www.wikidata.org/entity/Q6685514'), ('name', 'Lou Gehrig Memorial Award')]
[('x', 'http://www.wikidata.org/entity/Q1641970'), ('name', 'John R. Wooden Award')]
[('x', 'http://www.wikidata.or

In [15]:
### insert the result of TASK 8 in the file
og_uri = "http://www.wikidata.org/entity/Q222047"
og_name = "NBA Most Valuable Player Award"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"8", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_1.json
JSON object updated


`wd:Q222047` is the NBA Most Valuable Player Award

## Task 9

In [9]:
# starting from Lebron James I look at his properties 
queryString = """
SELECT ?player ?name
WHERE { 
    
    {
    select DISTINCT ?player where{
        ?team wdt:P118 wd:Q155223.
        ?player wdt:P54 ?team.
    }
    }
    ?prize wdt:P1346 ?player.
    FILTER(?prize = wd:Q222047) 
    ?player sc:name ?name.
}
ORDER BY ?player
"""

print("Results")
x = run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q134183'), ('name', 'Magic Johnson')]
[('player', 'http://www.wikidata.org/entity/Q136678'), ('name', 'James Harden')]
[('player', 'http://www.wikidata.org/entity/Q17717'), ('name', 'Oscar Robertson')]
[('player', 'http://www.wikidata.org/entity/Q179051'), ('name', 'Kareem Abdul-Jabbar')]
[('player', 'http://www.wikidata.org/entity/Q182455'), ('name', 'Wilt Chamberlain')]
[('player', 'http://www.wikidata.org/entity/Q189240'), ('name', 'Kevin Garnett')]
[('player', 'http://www.wikidata.org/entity/Q190094'), ('name', 'Allen Iverson')]
[('player', 'http://www.wikidata.org/entity/Q190152'), ('name', 'Larry Bird')]
[('player', 'http://www.wikidata.org/entity/Q192707'), ('name', 'Charles Barkley')]
[('player', 'http://www.wikidata.org/entity/Q192955'), ('name', 'Steve Nash')]
[('player', 'http://www.wikidata.org/entity/Q193020'), ('name', 'Tim Duncan')]
[('player', 'http://www.wikidata.org/entity/Q196185'), ('name', 'Russell Westbrook')]
[(

In [20]:
# starting from Lebron James I look at his properties 
queryString = """
SELECT ?player ?name COUNT(*) AS ?victories
WHERE { 
    {
        SELECT DISTINCT ?player{
            ?team wdt:P118 wd:Q155223.
            ?player wdt:P54 ?team.
        }
    }
    ?player wdt:P166 wd:Q222047.
    
    ?player sc:name ?name.
}
GROUP BY ?player ?name
"""

print("Results")
x = run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q264393'), ('name', 'Dave Cowens'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q368561'), ('name', 'Bob Pettit'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q205326'), ('name', 'Derrick Rose'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q17281073'), ('name', 'Nikola Jokić'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q242185'), ('name', 'Bill Russell'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q36159'), ('name', 'LeBron James'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q136678'), ('name', 'James Harden'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q190094'), ('name', 'Allen Iverson'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q312643'), ('name', 'Bob Cousy'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q553232'), ('name', 'Bob McAdoo'), ('victor

In [21]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    victories = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= victories
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"9", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_1.json
JSON object updated
